In [27]:
import psycopg2 as pg
import pandas as pd

In [15]:
# Credentials
host = 'comunidade-ds-postgres.c50pcakiuwi3.us-east-1.rds.amazonaws.com'
port = 5432
database = 'comunidadedsdb'
username = 'member'
password = 'cdspa'


In [37]:
# create variable to connect to db

conn = pg.connect( user = username,
            password = password,
            host = host,
            port = port,
            database = database)

# 1.0. Schema Query

In [12]:
# create a cursor
cursor = conn.cursor()

query_schema = """
    SELECT nspname
    FROM pg_catalog.pg_namespace

"""

cursor.execute( query_schema )
# extract query result
record = cursor.fetchall()
print(record)
# after finishing queries, always close connection!
cursor.close()
conn.close()

# 2.0. Table Query

In [18]:
# create a cursor
cursor = conn.cursor()

query_table = """
    SELECT tablename
    FROM pg_tables
    WHERE schemaname = 'pa004'

"""

cursor.execute( query_table )
# extract query result
record = cursor.fetchall()
print(record)
# after finishing queries, always close connection!
cursor.close()
conn.close()

[('users',), ('vehicle',), ('insurance',)]


# 3.0. Collect Data with Psycopg2

In [23]:
# create a cursor
cursor = conn.cursor()

query_table_users = """
    SELECT *
    FROM pa004.users u
    WHERE u.age > 44
    LIMIT 10

"""

cursor.execute( query_table_users )
# extract query result
record = cursor.fetchall()
print(record)
# after finishing queries, always close connection!
cursor.close()
conn.close()

[(2, 'Male', 76, 3.0, 26.0), (3, 'Male', 47, 28.0, 26.0), (8, 'Female', 56, 28.0, 26.0), (11, 'Female', 47, 35.0, 124.0), (14, 'Male', 76, 28.0, 13.0), (15, 'Male', 71, 28.0, 30.0), (20, 'Female', 60, 33.0, 124.0), (21, 'Male', 65, 28.0, 124.0), (22, 'Male', 49, 28.0, 124.0), (27, 'Female', 51, 28.0, 124.0)]


In [28]:
data = pd.DataFrame( record )

# bom mas nao preenche colunas direto
data

# 4.0. Collect Data with Pandas
To import db into DFs, the best way is to use pandas

In [30]:
import pandas.io.sql as psql

In [39]:
query_table_users = """
    SELECT *
    FROM pa004.users u LEFT JOIN pa004.vehicle v ON (u.id = v.id)
                       LEFT JOIN pa004.insurance i ON (u.id = i.id)
    WHERE u.age > 44

"""
df = psql.read_sql(query_table_users, conn)
df

,id,gender,age,region_code,policy_sales_channel,id,driving_license,vehicle_age,vehicle_damage,id,previously_insured,annual_premium,vintage,response
0,8,Female,56,28.0,26.0,8,1,1-2 Year,Yes,8,0,32031.0,72,1
1,11,Female,47,35.0,124.0,11,1,1-2 Year,Yes,11,0,47576.0,46,1
2,39,Male,45,8.0,124.0,39,1,1-2 Year,Yes,39,0,42297.0,264,0
3,42,Female,54,28.0,122.0,42,1,1-2 Year,No,42,0,38560.0,184,0
4,53,Female,49,3.0,152.0,53,1,1-2 Year,No,53,1,25553.0,130,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131943,381078,Female,45,2.0,152.0,381078,1,1-2 Year,No,381078,1,34003.0,288,0
131944,381079,Female,58,28.0,26.0,381079,1,1-2 Year,Yes,381079,0,39740.0,170,0
131945,381082,Female,75,28.0,26.0,381082,1,1-2 Year,No,381082,1,51619.0,80,0
131946,381097,Female,49,10.0,26.0,381097,1,1-2 Year,Yes,381097,0,34541.0,44,0


In [40]:
query_table_users = """
    SELECT *
    FROM pa004.users u LEFT JOIN pa004.vehicle v ON (u.id = v.id)
                       LEFT JOIN pa004.insurance i ON (u.id = i.id)
    WHERE u.age > 44
    """

df = pd.read_sql(query_table_users, conn)
df = df.drop(df.columns[[5, 9 ]], axis = 1)
df

,gender,age,region_code,policy_sales_channel,driving_license,vehicle_age,vehicle_damage,previously_insured,annual_premium,vintage,response
0,Female,56,28.0,26.0,1,1-2 Year,Yes,0,32031.0,72,1
1,Female,47,35.0,124.0,1,1-2 Year,Yes,0,47576.0,46,1
2,Male,45,8.0,124.0,1,1-2 Year,Yes,0,42297.0,264,0
3,Female,54,28.0,122.0,1,1-2 Year,No,0,38560.0,184,0
4,Female,49,3.0,152.0,1,1-2 Year,No,1,25553.0,130,0
...,...,...,...,...,...,...,...,...,...,...,...
131943,Female,45,2.0,152.0,1,1-2 Year,No,1,34003.0,288,0
131944,Female,58,28.0,26.0,1,1-2 Year,Yes,0,39740.0,170,0
131945,Female,75,28.0,26.0,1,1-2 Year,No,1,51619.0,80,0
131946,Female,49,10.0,26.0,1,1-2 Year,Yes,0,34541.0,44,0
